In [ ]:
import tweepy
import csv
import pandas as pd
import psycopg2

# Connect to DB

consumer_key = 'KfMKNmydahwQSubsT297DpbYt'
consumer_secret = 'ccCdXjEEM1vPsuA4sgJNAOTwW9EueZHejhBe42VjO2brWexLdt'
access_token = '819912293521772548-hE5XFuCPGIh5K6GWKp541zwf06YJ4sg'
access_token_secret = 'RQahNZKvkS5iVFr5bKF1PI9HpW6yxmhWyPI5weGKU5yaR'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

try:
    conn = psycopg2.connect("dbname=eksploracja_danych user=postgres password=postgres", host="localhost")
    print "Connected to the database"
except:
    print "I am unable to connect to the database"
    
cur = conn.cursor()

# Try on connection

try:
    cur.execute("""SELECT id, phrase FROM hashtag""")
    cur.execute("""SELECT author_id, tweet_id, date FROM re_tweet""")
    cur.execute("""SELECT id, name FROM author""")
    cur.execute("""SELECT id, author_id, searched_hashtag_id, date, content from tweet""")
    print "Properly set all tables"
except:
    print "I can't properly commit transactions to DB"

In [ ]:
# Prepare download

insert_tag_sql = "INSERT INTO hashtag (phrase) VALUES (%s) RETURNING id"
insert_retweet_sql = "INSERT INTO re_tweet (author_id, tweet_id, date) VALUES (%s, %s, %s)"
insert_user_sql = "INSERT INTO author (id, name) VALUES (%s, %s)"
insert_tweet_sql = "INSERT INTO tweet (id, author_id, searched_hashtag_id, date, content) VALUES (%s, %s, %s, %s, %s)"


tags = [
#      "#WspólnieTworzymyPrzyszłośćKrakowa"
#     "#NowyPrezydentWiekszeMozliwości"
#       "#wybory"
     "#Wybory2018"
#     "#WyborySamorzadowe"
#      "#wyborysamorzadowe2018"
       ]

In [ ]:
for tag in tags:
    csvFile = open(tag+ '.csv', 'a')
    csvWriter = csv.writer(csvFile)

    cur.execute(insert_tag_sql, (tag,))
    conn.commit()
    tag_id = cur.fetchone()[0]
    print(tag)

    for tweet in tweepy.Cursor(api.search,q=tag,
                           lang="pl",
                           since="2018-10-28",
                           until="2018-11-08").items():
#         print tweet.text.encode('utf-8')
#         print tweet.retweet_count    
#         print tweet.author.__dict__.keys()
#         print "\n\n\n"

        if hasattr(tweet, 'retweeted_status') or "RT @" in tweet.text.encode('utf-8'):
#             print('retweet')
            cur.execute(insert_retweet_sql, (tweet.author.id, tweet.id, tweet.created_at))
            conn.commit()
        else:
            print(tweet.created_at)
            try:
                cur.execute(insert_tweet_sql, (tweet.id, tweet.author.id, tag_id,  tweet.created_at, tweet.text.encode('utf-8')))
            except:
                print("this tweet already exists in db")
            try:
                cur.execute(insert_user_sql, (tweet.author.id, tweet.author.name))
            except:
                print("this author exists")
            conn.commit()

        csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])
    
    print "TAG END"

